In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego as mplego
import neuropacks
import numpy as np
import os

from pyprojroot import here
from scipy.stats import wilcoxon
from mpl_lego.labels import add_significance_label, bold_text

%matplotlib inline

In [ ]:
mplego.style.use_latex_style()

In [ ]:
data_root = here('data')

In [ ]:
# Import PVC data
pvc_data_path = os.path.join(data_root, 'data_monkey1_gratings.mat')
pvc11 = neuropacks.PVC11(data_path=pvc_data_path)

In [ ]:
# Import PVC fits
pvc_cotu_results = h5py.File(here('fits/pvc11_cotu_fits.h5'), 'r')
pvc_cotula_results = h5py.File(here('fits/pvc11_cotula_fits.h5'), 'r')
a_pvc_cotu = np.mean(pvc_cotu_results['tc_cosine2_Lasso/coupling_coefs'], axis=0)
a_pvc_cotula = np.array(pvc_cotula_results['a'][:])
b_pvc_cotu = np.mean(pvc_cotu_results['tc_cosine2_Lasso/tuning_coefs'], axis=0)
b_pvc_cotula = np.array(pvc_cotula_results['b'][:])
modulations_pvc_cotula, _ = pvc11.get_tuning_modulation_and_preference(form='cosine2', tuning_coefs=b_pvc_cotula)

In [ ]:
n_coupling = a_pvc_cotula.shape[0]
a_pvc_cotula_full = np.zeros((n_coupling, n_coupling))
a_pvc_cotu_full = np.zeros((n_coupling, n_coupling))
a_pvc_cotula_full[np.not_equal.outer(np.arange(n_coupling), np.arange(n_coupling))] = a_pvc_cotula.ravel()
a_pvc_cotu_full[np.not_equal.outer(np.arange(n_coupling), np.arange(n_coupling))] = a_pvc_cotu.ravel()

In [ ]:
X = pvc11.get_design_matrix(form='cosine2')
Y = pvc11.get_response_matrix(transform='square_root')

In [ ]:
b_pvc_cotu_var = np.mean((X @ b_pvc_cotu.T)**2, axis=0) / np.mean(Y**2, axis=0)
b_pvc_cotula_var = np.mean((X @ b_pvc_cotula.T)**2, axis=0) / np.mean(Y**2, axis=0)
a_pvc_cotu_var = np.mean((Y @ a_pvc_cotu_full.T)**2, axis=0) / np.mean(Y**2, axis=0)
a_pvc_cotula_var = np.mean((Y @ a_pvc_cotula_full.T)**2, axis=0) / np.mean(Y**2, axis=0)
l_pvc_cotula_var = np.mean(np.mean(pvc_cotula_results['L'][:, :, 0]**2, axis=1)) / np.mean(Y**2, axis=0)

In [ ]:
# Import ECOG data
ecog_data_path = os.path.join(data_root, 'r32_b7.mat')
ecog_grid_path = os.path.join(data_root, 'grdid.mat')
ecog = neuropacks.ECOG(data_path=ecog_data_path, grid_path=ecog_grid_path)

In [ ]:
X = ecog.get_design_matrix()
Y = ecog.get_response_matrix(
    bounds=(40, 60),
    band='HG',
    electrodes=None,
    transform=None)
Y_centered = Y - Y.mean(axis=0)

In [ ]:
# Import ECOG fits
ecog_cotu_fits = h5py.File(here('fits/ecog_cotu_fits.h5'), 'r')
ecog_cotula_fits = h5py.File(here('fits/ecog_cotula_fits.h5'), 'r')
a_ecog_cotu = np.mean(ecog_cotu_fits['tc_bf_lasso/coupling_coefs'], axis=0)
a_ecog_cotula = np.array(ecog_cotula_fits['a'][:])
b_ecog_cotu = np.mean(ecog_cotu_fits['tc_bf_lasso/tuning_coefs'], axis=0)
b_ecog_cotula = np.array(ecog_cotula_fits['b'][:])
n_electrodes = b_ecog_cotula.shape[0]
modulations_ecog_cotula = np.array(
    [ecog.get_tuning_modulation_and_preference(tuning_coefs=b_ecog_cotula[idx], form='bf')[0]
     for idx in range(n_electrodes)]).ravel()

In [ ]:
n_coupling = a_ecog_cotula.shape[0]
a_ecog_cotula_full = np.zeros((n_coupling, n_coupling))
a_ecog_cotu_full = np.zeros((n_coupling, n_coupling))
a_ecog_cotula_full[np.not_equal.outer(np.arange(n_coupling), np.arange(n_coupling))] = a_ecog_cotula.ravel()

In [ ]:
b_ecog_cotu_var = np.mean((X @ b_ecog_cotu.T - (X @ b_ecog_cotu.T).mean(axis=0))**2, axis=0) / np.mean(Y**2, axis=0)
b_ecog_cotula_var = np.mean((X @ b_ecog_cotula.T - (X @ b_ecog_cotula.T).mean(axis=0))**2, axis=0) / np.mean(Y**2, axis=0)
a_ecog_cotu_var = np.mean((Y @ a_ecog_cotu.T - (Y @ a_ecog_cotu.T).mean(axis=0))**2, axis=0) / np.mean(Y**2, axis=0)
a_ecog_cotula_var = np.mean((Y @ a_ecog_cotula_full.T - (Y @ a_ecog_cotula_full.T).mean(axis=0))**2, axis=0) / np.mean(Y**2, axis=0)
l_ecog_cotula_var = np.mean(np.mean(ecog_cotula_fits['L'][:, :, 0]**2, axis=1)) / np.mean(Y**2, axis=0)

In [ ]:
"""
Figure 4
"""
width = 6
height = 10
fig = plt.figure(figsize=(width, height))

# Preset colors
tu_model_color = 'black'
cotu_model_color = 'gray'
cotula_model_color = 'red'
# Tuning plot settings
tuning_curve_lw = 2
tuning_plot_label_size = 15
# Scatterplot settings
scatter_marker = 'o'
scatter_face_color = 'black'
scatter_edge_color = 'white'
scatter_point_size = 50
scatter_alpha = 0.5
scatter_label_size = 18
scatter_xlabel_pad = 2
scatter_tick_size = 15
# Legend settings
legend_handletextpad = 0.5
legend_handlelength = 1
legend_columnspacing = 0.7
legend_size = 13
# Identity line settings
identity_line_color = 'gray'
identity_line_lw = 3
identity_line_alpha = 0.5
# Inset inset settings
box_width = 0
box_lw = 3
box_marker = 'o'
box_whisker_lw = 0
box_showfliers = False
inset_label_size = 13
inset_tick_size = 12
inset_sig_size = 15
# Subplot label settings
subplot_label_size = 20
subplot_label_bold = True

"""
Figure 4a:
Plot tuning curves, PVC fits
"""
# Figure dimensions
pvc_tcurve_w = 1
pvc_tcurve_h = 1
pvc_tcurve_x = 0
pvc_tcurve_y_base = 0

# Choose example neurons
neuron_ids = np.array([49, 23, 103, 43])

# Iterate over example neurons
for n_idx, neuron in enumerate(neuron_ids):
    # Create axis object
    pvc_tuning_plot = fig.add_axes([pvc_tcurve_x / width,
                                    pvc_tcurve_y_base / height + n_idx * (pvc_tcurve_w / height),
                                    pvc_tcurve_w / width,
                                    pvc_tcurve_h / height])

    # Calculate tuning curves
    angles, pvc_tu_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=np.mean(pvc_cotu_results['t_cosine2/tuning_coefs'], axis=0)[neuron])
    angles, pvc_cotu_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=np.mean(pvc_cotu_results['tc_cosine2_Lasso/tuning_coefs'], axis=0)[neuron])
    angles, pvc_cotula_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=b_pvc_cotula[neuron])
    # Plot tuning curves
    pvc_tuning_plot.plot(
        angles,
        pvc_tu_tuning,
        color=tu_model_color,
        linewidth=tuning_curve_lw)
    pvc_tuning_plot.plot(
        angles,
        pvc_cotu_tuning,
        color=cotu_model_color,
        linewidth=tuning_curve_lw)
    pvc_tuning_plot.plot(
        angles,
        pvc_cotula_tuning,
        color=cotula_model_color,
        linewidth=tuning_curve_lw)
    # Set axis bounds
    pvc_tuning_plot.set_xlim([0, 180])
    pvc_tuning_plot.set_xticks([])
    pvc_tuning_plot.set_yticks([])
    # Insert labels
    if n_idx == 0:
        pvc_tuning_plot.set_xlabel(bold_text('Angle'), fontsize=tuning_plot_label_size)
        pvc_tuning_plot.set_ylabel(bold_text('Response'), fontsize=tuning_plot_label_size)

"""
Figure 4b:
Scatter plot of modulations, PVC fits
"""
# Figure dimensions
pvc_scatter_w = 4
pvc_scatter_h = 4
pvc_scatter_pad = 0.95
pvc_scatter_x = pvc_tcurve_x + pvc_tcurve_w + pvc_scatter_pad
pvc_scatter_y = 0

# Create axis object
pvc_scatter = fig.add_axes([pvc_scatter_x / width,
                            pvc_scatter_y / height,
                            pvc_scatter_w / width,
                            pvc_scatter_h / height])

# Plot modulations
pvc_scatter.scatter(
    pvc_cotu_results['tc_cosine2_Lasso/modulations_mean'][:],
    modulations_pvc_cotula,
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)

# Plot placeholders for labels
pvc_scatter.plot([], color=tu_model_color, label=bold_text('Tuning Model'))
pvc_scatter.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
pvc_scatter.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

# Create legend
lgd = pvc_scatter.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

# Identity line 
pvc_scatter.plot([0.01, 10], [0.01, 10],
                 color=identity_line_color,
                 linewidth=identity_line_lw,
                 alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
pvc_scatter.set_xlim([0.01, 10])
pvc_scatter.set_ylim([0.01, 10])
pvc_scatter.set_xticks([0.01, 0.1, 1, 10])
pvc_scatter.set_yticks([0.01, 0.1, 1, 10])
pvc_scatter.set_xscale('log')
pvc_scatter.set_yscale('log')
# Axis labels
pvc_scatter.set_xlabel(bold_text('CoTu Model'),
                       fontsize=scatter_label_size,
                       labelpad=scatter_xlabel_pad)
pvc_scatter.set_ylabel(bold_text('CoTuLa Model'),
                       fontsize=scatter_label_size)
pvc_scatter.tick_params(labelsize=scatter_tick_size)

"""
Figure 4b, inset:
Boxplot of modulations
"""
# Figure dimensions
pvc_modulations_w = 1.
pvc_modulations_h = 1.2
pvc_modulations_pad = 2.90
pvc_modulations_x = pvc_scatter_x + pvc_modulations_pad
pvc_modulations_y_pad = 0.1
pvc_modulations_y = pvc_scatter_y + pvc_modulations_y_pad

pvc_mod_ax = fig.add_axes([pvc_modulations_x / width,
                           pvc_modulations_y / height,
                           pvc_modulations_w / width,
                           pvc_modulations_h / height])
# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
for model_idx, model in enumerate([pvc_cotu_results['t_cosine2/modulations_mean'][:],
                                   pvc_cotu_results['tc_cosine2_Lasso/modulations_mean'][:],
                                   modulations_pvc_cotula]):
    pvc_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers,
        whis=0)
# Set axis limits, ticks, and scale
pvc_mod_ax.set_xlim([-0.5, 2.5])
pvc_mod_ax.set_ylim([0.1, 10])
pvc_mod_ax.set_xticks([])
pvc_mod_ax.set_yticks([0.1, 1, 10])
pvc_mod_ax.set_yscale('log')
pvc_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
pvc_mod_ax.set_ylabel(bold_text('Modulations'), fontsize=inset_label_size)
# Add significance labels
add_significance_label(pvc_mod_ax,
                       bounds=[1, 2],
                       label='***',
                       fontsize=inset_sig_size)
add_significance_label(pvc_mod_ax,
                       bounds=[0, 2],
                       label='***',
                       fontsize=inset_sig_size,
                       spacing=0.2)

"""
Figure 4c:
Scatter plot of coupling modulations, PVC fits
"""
# Figure dimensions
pvc_coupling_pad = pvc_scatter_pad * 1.2
pvc_coupling_scatter_x = pvc_scatter_x + pvc_scatter_w + pvc_coupling_pad
pvc_coupling_scatter_y = pvc_scatter_y

# Create axis object
pvc_coupling = fig.add_axes([pvc_coupling_scatter_x / width,
                             pvc_coupling_scatter_y / height,
                             pvc_scatter_w / width,
                             pvc_scatter_h / height])

# Plot modulations
pvc_coupling.scatter(
    a_pvc_cotu_full.ravel(),
    a_pvc_cotula_full.ravel(),
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)

# Identity line 
pvc_coupling.plot([-0.3, 0.5], [-0.3, 0.5],
                   color=identity_line_color,
                   linewidth=identity_line_lw,
                   alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
pvc_coupling.set_xlim([-0.3, 0.5])
pvc_coupling.set_ylim([-0.3, 0.5])
pvc_coupling.set_xticks([-0.2, 0.0, 0.2, 0.4])
pvc_coupling.set_yticks([-0.2, 0.0, 0.2, 0.4])

# Axis labels
pvc_coupling.set_xlabel(bold_text('CoTu Model'),
                        fontsize=scatter_label_size,
                        labelpad=scatter_xlabel_pad)
pvc_coupling.set_ylabel(bold_text('CoTuLa Model'),
                        fontsize=scatter_label_size)
pvc_coupling.tick_params(labelsize=scatter_tick_size)

# Plot placeholders for labels
pvc_coupling.plot([], color=tu_model_color, label=bold_text('Coupling Model'))
pvc_coupling.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
pvc_coupling.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

# Create legend
lgd = pvc_coupling.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

"""
Figure 4c, inset:
Boxplot of coupling modulations
"""
# Figure dimensions
pvc_coupling_modulations_x = pvc_coupling_scatter_x + pvc_modulations_pad
pvc_coupling_modulations_y = pvc_coupling_scatter_y + pvc_modulations_y_pad

pvc_coupling_mod_ax = fig.add_axes([pvc_coupling_modulations_x / width,
                                    pvc_coupling_modulations_y / height,
                                    pvc_modulations_w / width,
                                    pvc_modulations_h / height])

# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
c_coefs = np.median(pvc_cotu_results['c_UoI_Lasso/coupling_coefs'][:], axis=0).ravel()
nz_coefs = c_coefs[c_coefs != 0]

for model_idx, model in enumerate([nz_coefs,
                                   a_pvc_cotu_full.ravel()[a_pvc_cotu_full.ravel() != 0],
                                   a_pvc_cotula_full.ravel()[a_pvc_cotula_full.ravel() != 0]]):
    pvc_coupling_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers,
        whis=0)

pvc_coupling_mod_ax.set_xlim([-0.5, 2.5])
pvc_coupling_mod_ax.set_ylim([-0.1, 0.1])
pvc_coupling_mod_ax.set_xticks([])
pvc_coupling_mod_ax.set_yticks([-0.1, 0, 0.1])
pvc_coupling_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
pvc_coupling_mod_ax.set_ylabel(bold_text('Coupling Coefs'), fontsize=inset_label_size)

"""
Figure 4d:
Plot tuning curves, ECOG fits
"""
# Figure dimensions
ecog_tcurve_w = 1
ecog_tcurve_h = 1
ecog_tcurve_pad = 1
ecog_tcurve_x = 0
ecog_tcurve_y_pad = 1
ecog_tcurve_y_base = -4 * ecog_tcurve_h - ecog_tcurve_y_pad

# Choose example electrodes
electrode_ids = np.array([59, 77, 26, 62])

# Iterate over example electrodes
for e_idx, electrode in enumerate(electrode_ids):
    # Create axis object
    ecog_tuning_plot = fig.add_axes([ecog_tcurve_x / width,
                                     ecog_tcurve_y_base / height + e_idx * (ecog_tcurve_w / height),
                                     ecog_tcurve_w / width,
                                     ecog_tcurve_h / height])
    # Calculate tuning curves
    frequencies, ecog_tu_tuning = ecog.get_tuning_curve(
        tuning_coefs=np.mean(ecog_cotu_fits['t_bf_lasso/tuning_coefs'], axis=0)[electrode])
    frequencies, ecog_cotu_tuning = ecog.get_tuning_curve(
        tuning_coefs=np.mean(ecog_cotu_fits['tc_bf_lasso/tuning_coefs'], axis=0)[electrode])
    frequencies, ecog_cotula_tuning = ecog.get_tuning_curve(
        tuning_coefs=b_ecog_cotula[electrode])
    # Plot tuning curves
    ecog_tuning_plot.plot(
        frequencies,
        ecog_tu_tuning.ravel() - ecog_tu_tuning.ravel().min(),
        color=tu_model_color,
        linewidth=tuning_curve_lw)
    ecog_tuning_plot.plot(
        frequencies,
        ecog_cotu_tuning.ravel() - ecog_cotu_tuning.ravel().min(),
        color=cotu_model_color,
        linewidth=tuning_curve_lw)
    ecog_tuning_plot.plot(
        frequencies,
        ecog_cotula_tuning.ravel() - ecog_cotula_tuning.ravel().min(),
        color=cotula_model_color,
        linewidth=tuning_curve_lw)
    
    # Set axis bounds
    ecog_tuning_plot.set_xlim([400, 34000])
    ecog_tuning_plot.set_xscale('log')
    ecog_tuning_plot.set_xticks([])
    ecog_tuning_plot.set_yticks([])
    
    # Set axis labels
    if e_idx == 0:
        ecog_tuning_plot.set_xlabel(bold_text('Log Freq.'), fontsize=tuning_plot_label_size)
        ecog_tuning_plot.set_ylabel(bold_text('Response'), fontsize=tuning_plot_label_size)

"""
Figure 4d:
Scatter plot of modulations, ECOG fits
"""
# Figure dimensions
ecog_scatter_w = 4
ecog_scatter_h = 4
ecog_scatter_pad = pvc_scatter_pad
ecog_scatter_x = ecog_tcurve_x + ecog_tcurve_w + ecog_scatter_pad
ecog_scatter_y = ecog_tcurve_y_base

# Create axis object
ecog_scatter = fig.add_axes([ecog_scatter_x / width,
                             ecog_scatter_y / height,
                             ecog_scatter_w / width,
                             ecog_scatter_h / height])

# Plot modulations
ecog_scatter.scatter(
    np.mean(ecog_cotu_fits['tc_bf_lasso/modulations'][:], axis=0),
    modulations_ecog_cotula,
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)

# Plot placeholders for labels
ecog_scatter.plot([], color=tu_model_color, label=bold_text('Tuning Model'))
ecog_scatter.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
ecog_scatter.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

# Create legend
lgd = ecog_scatter.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

# Identity line 
ecog_scatter.plot([0.001, 10], [0.001, 10],
                  color=identity_line_color,
                  linewidth=identity_line_lw,
                  alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
ecog_scatter.set_xlim([0.001, 10])
ecog_scatter.set_ylim([0.001, 10])
ecog_scatter.set_xticks([0.001, 0.01, 0.1, 1, 10])
ecog_scatter.set_yticks([0.001, 0.01, 0.1, 1, 10])
ecog_scatter.set_xscale('log')
ecog_scatter.set_yscale('log')
# Axis labels
ecog_scatter.set_xlabel(bold_text('CoTu Model'),
                        fontsize=scatter_label_size,
                        labelpad=scatter_xlabel_pad)
ecog_scatter.set_ylabel(bold_text('CoTuLa Model'),
                        fontsize=scatter_label_size)
ecog_scatter.tick_params(labelsize=scatter_tick_size)

"""
Figure 4b, inset:
Boxplot of modulations
"""
# Figure dimensions
ecog_modulations_w = 1.
ecog_modulations_h = 1.2
ecog_modulations_pad = pvc_modulations_pad
ecog_modulations_x = ecog_scatter_x + ecog_modulations_pad
ecog_modulations_y_pad = pvc_modulations_y_pad
ecog_modulations_y = ecog_scatter_y + ecog_modulations_y_pad

ecog_mod_ax = fig.add_axes([ecog_modulations_x / width,
                            ecog_modulations_y / height,
                            ecog_modulations_w / width,
                            ecog_modulations_h / height])
# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
for model_idx, model in enumerate([np.mean(ecog_cotu_fits['t_bf_lasso/modulations'][:], axis=0),
                                   np.mean(ecog_cotu_fits['tc_bf_lasso/modulations'][:], axis=0),
                                   modulations_ecog_cotula]):
    ecog_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers,
        whis=0)
# Set axis limits, ticks, and scale
ecog_mod_ax.set_xlim([-0.5, 2.5])
ecog_mod_ax.set_ylim([0.01, 10])
ecog_mod_ax.set_xticks([])
ecog_mod_ax.set_yticks([0.01, 0.1, 1, 10])
ecog_mod_ax.set_yscale('log')
ecog_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
ecog_mod_ax.set_ylabel(bold_text('Modulations'), fontsize=inset_label_size)
# Add significance labels
add_significance_label(ecog_mod_ax,
                       bounds=[1, 2],
                       label='***',
                       fontsize=inset_sig_size)
add_significance_label(ecog_mod_ax,
                       bounds=[0, 2],
                       label='***',
                       fontsize=inset_sig_size,
                       spacing=0.2)

"""
Figure 4d:
Scatter plot of coupling modulations, ECOG fits
"""
# Figure dimensions
ecog_coupling_pad = ecog_scatter_pad * 1.2
ecog_coupling_scatter_x = ecog_scatter_x + ecog_scatter_w + ecog_coupling_pad
ecog_coupling_scatter_y = ecog_scatter_y

# Create axis object
ecog_coupling = fig.add_axes([ecog_coupling_scatter_x / width,
                              ecog_coupling_scatter_y / height,
                              ecog_scatter_w / width,
                              ecog_scatter_h / height])

# Plot modulations
ecog_coupling.scatter(
    a_ecog_cotu.ravel(),
    a_ecog_cotula_full.ravel(),
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)

# Identity line 
ecog_coupling.plot([-0.6, 1.2], [-0.6, 1.2],
                   color=identity_line_color,
                   linewidth=identity_line_lw,
                   alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
ecog_coupling.set_xlim([-0.6, 1.2])
ecog_coupling.set_ylim([-0.6, 1.2])
ecog_coupling.set_xticks([-0.5, 0.0, 0.5, 1.0])
ecog_coupling.set_yticks([-0.5, 0.0, 0.5, 1.0])

# Axis labels
ecog_coupling.set_xlabel(bold_text('CoTu Model'),
                         fontsize=scatter_label_size,
                         labelpad=scatter_xlabel_pad)
ecog_coupling.set_ylabel(bold_text('CoTuLa Model'),
                         fontsize=scatter_label_size)
ecog_coupling.tick_params(labelsize=scatter_tick_size)

# Plot placeholders for labels
ecog_coupling.plot([], color=tu_model_color, label=bold_text('Coupling Model'))
ecog_coupling.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
ecog_coupling.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

lgd = ecog_coupling.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

"""
Figure 4f, inset:
Boxplot of coupling modulations
"""
# Figure dimensions
ecog_coupling_modulations_x = ecog_coupling_scatter_x + ecog_modulations_pad
ecog_coupling_modulations_y = ecog_coupling_scatter_y + ecog_modulations_y_pad

ecog_coupling_mod_ax = fig.add_axes([ecog_coupling_modulations_x / width,
                                     ecog_coupling_modulations_y / height,
                                     ecog_modulations_w / width,
                                     ecog_modulations_h / height])

# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
c_coefs = np.median(ecog_cotu_fits['c_lasso/coupling_coefs'][:], axis=0).ravel()
nz_coefs = c_coefs[c_coefs != 0]

for model_idx, model in enumerate([nz_coefs,
                                   a_ecog_cotu.ravel()[a_ecog_cotu.ravel() != 0],
                                   a_ecog_cotula_full.ravel()[a_ecog_cotula_full.ravel() != 0]]):
    ecog_coupling_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers,
        whis=0)

ecog_coupling_mod_ax.set_xlim([-0.5, 2.5])
ecog_coupling_mod_ax.set_ylim([-0.05, 0.05])
ecog_coupling_mod_ax.set_xticks([])
ecog_coupling_mod_ax.set_yticks([-0.05, 0, 0.05])
ecog_coupling_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
ecog_coupling_mod_ax.set_ylabel(bold_text('Coupling Coefs'), fontsize=inset_label_size)


"""
Figure 4g:
Variance fraction
"""
# Figure dimensions
var_frac_y_pad = -3.5
var_frac_x = 0
var_frac_y = ecog_tcurve_y_base + var_frac_y_pad
var_frac_w = 6
var_frac_h = 2

var_frac_ax = fig.add_axes([var_frac_x / width,
                            var_frac_y / height,
                            var_frac_w / width,
                            var_frac_h / height])

a_pvc_cotu_var_mean = a_pvc_cotu_var.mean()
b_pvc_cotu_var_mean = b_pvc_cotu_var.mean()
err_pvc_cotu_var_mean = 1 - a_pvc_cotu_var_mean - b_pvc_cotu_var_mean

a_pvc_cotula_var_mean = a_pvc_cotula_var.mean()
b_pvc_cotula_var_mean = b_pvc_cotula_var.mean()
l_pvc_cotula_var_mean = l_pvc_cotula_var.mean()
err_pvc_cotula_var_mean = 1 - a_pvc_cotula_var_mean - b_pvc_cotula_var_mean - l_pvc_cotula_var_mean

a_ecog_cotu_var_mean = a_ecog_cotu_var.mean()
b_ecog_cotu_var_mean = b_ecog_cotu_var.mean()
err_ecog_cotu_var_mean = 1 - a_ecog_cotu_var_mean - b_ecog_cotu_var_mean

a_ecog_cotula_var_mean = a_ecog_cotula_var.mean()
b_ecog_cotula_var_mean = b_ecog_cotula_var.mean()
l_ecog_cotula_var_mean = l_ecog_cotula_var.mean()
err_ecog_cotula_var_mean = 1 - a_ecog_cotula_var_mean - b_ecog_cotula_var_mean - l_ecog_cotula_var_mean


# PVC, Coupling
var_frac_ax.barh(
    y=[4, 3],
    width=[a_pvc_cotula_var_mean, a_pvc_cotu_var_mean],
    color='#B5CAF6',
    height=0.8,
    label='Co')
# PVC, Tuning
var_frac_ax.barh(
    y=[4, 3],
    width=[a_pvc_cotula_var_mean + b_pvc_cotula_var_mean, a_pvc_cotu_var_mean + b_pvc_cotu_var_mean],
    color='#D03039',
    zorder=-1,
    height=0.8,
    label='Tu')
# PVC, Latent
var_frac_ax.barh(
    y=[4, 3],
    width=[a_pvc_cotula_var_mean + b_pvc_cotula_var_mean + l_pvc_cotula_var_mean, a_pvc_cotu_var_mean + b_pvc_cotu_var_mean],
    color='#22A499',
    zorder=-2,
    height=0.8,
    label='La')
# PVC, Error
var_frac_ax.barh(
    y=[4, 3],
    width=[1, 1],
    color='#C9C9C9',
    zorder=-3,
    height=0.8,
    label='Err')
var_frac_ax.text(x=a_pvc_cotula_var_mean + b_pvc_cotula_var_mean + 0.5 * l_pvc_cotula_var_mean,
                 y=4.6,
                 ha='center',
                 color='#22A499',
                 s='0.013',
                 fontsize=16)
var_frac_ax.text(x=0.01, y=4, s=bold_text('CoTuLa'), fontsize=16, ha='left', va='center')
var_frac_ax.text(x=0.01, y=3, s=bold_text('CoTu'), fontsize=16, ha='left', va='center')


var_frac_ax.barh(
    y=[1, 0],
    width=[a_ecog_cotula_var_mean, a_ecog_cotu_var_mean],
    color='#B5CAF6',
    height=0.8)
var_frac_ax.barh(
    y=[1, 0],
    width=[a_ecog_cotula_var_mean + 5 * b_ecog_cotula_var_mean, a_ecog_cotu_var_mean + 5 * b_ecog_cotu_var_mean],
    color='#D03039',
    zorder=-1,
    height=0.8)
var_frac_ax.barh(
    y=[1, 0],
    width=[a_ecog_cotula_var_mean + 5 * b_ecog_cotula_var_mean + 5 * l_ecog_cotula_var_mean, a_ecog_cotu_var_mean + 5 * b_ecog_cotu_var_mean],
    color='#22A499',
    zorder=-2,
    height=0.8)
# PVC, Error
var_frac_ax.barh(
    y=[1, 0],
    width=[1, 1],
    color='#C9C9C9',
    zorder=-3,
    height=0.8)

var_frac_ax.text(x=a_ecog_cotula_var_mean + b_ecog_cotula_var_mean,
                 y=1.5,
                 ha='right',
                 color='#D03039',
                 s='0.0012',
                 fontsize=16)
var_frac_ax.text(x=a_ecog_cotula_var_mean + 5 * b_ecog_cotula_var_mean + 5 * l_ecog_cotula_var_mean,
                 y=1.5,
                 ha='left',
                 color='#22A499',
                 s='0.0014',
                 fontsize=16)
var_frac_ax.text(x=a_ecog_cotula_var_mean + b_ecog_cotula_var_mean,
                 y=-0.9,
                 ha='center',
                 color='#D03039',
                 s=r'$9.5\times10^{-5}$',
                 fontsize=16)

var_frac_ax.set_xlim([0, 1.05])
var_frac_ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
var_frac_ax.set_yticks([0.5, 3.5])
var_frac_ax.set_ylim([-1.2, 5])
var_frac_ax.set_yticklabels(bold_text(['ECoG', 'PVC']), fontsize=20, rotation=90, va='center')
var_frac_ax.tick_params(labelsize=scatter_tick_size)
var_frac_ax.spines['top'].set_visible(False)
var_frac_ax.spines['right'].set_visible(False)
var_frac_ax.legend(loc='center left', bbox_to_anchor=(0.01, 1.2), ncol=4, prop={'size': 14})

"""
Figure 4h:
Variance fraction
"""
# Figure dimensions
var_comp_x = ecog_coupling_scatter_x
var_comp_y = var_frac_y
var_comp_w = 1.
var_comp_h = var_frac_h

var_comp_ax = fig.add_axes([var_comp_x / width,
                            var_comp_y / height,
                            var_comp_w / width,
                            var_comp_h / height])

var_comp_ax.boxplot(x=[b_ecog_cotu_var / a_ecog_cotu_var,
                       b_ecog_cotula_var / a_ecog_cotula_var],
                    positions=[0, 1],
                    widths=box_width,
                    boxprops={'linewidth': box_lw, 'color': 'black'},
                    medianprops={'marker': box_marker, 'markersize': 7, 'color': 'black'},
                    whiskerprops={'linewidth': box_whisker_lw, 'color': 'black'},
                    capprops={'linewidth': 2},
                    showcaps=True,
                    showfliers=box_showfliers,
                    whis=0)
var_comp_ax.set_yscale('log')
var_comp_ax.set_yticks([1e-5, 1e-4, 1e-3])
var_comp_ax.set_ylim([1e-5, 3e-3])
var_comp_ax.set_xticks([0, 1])
var_comp_ax.tick_params(labelsize=15)
var_comp_ax.set_xticklabels(bold_text(['CoTu', 'CoTuLa']), fontsize=15, ha='left', rotation=-25)
var_comp_ax.spines['top'].set_visible(False)
var_comp_ax.spines['right'].set_visible(False)
var_comp_ax.set_title(bold_text('ECoG'), fontsize=15, pad=35)

# Figure dimensions
var_comp_x_pad = 1
var_comp2_x = var_comp_x + var_comp_w + var_comp_x_pad
var_comp2_y = var_comp_y
var_comp2_w = 1.
var_comp2_h = var_frac_h

var_comp2_ax = fig.add_axes([var_comp2_x / width,
                             var_comp2_y / height,
                             var_comp2_w / width,
                             var_comp2_h / height])

var_comp2_ax.boxplot(x=[b_pvc_cotu_var / a_pvc_cotu_var,
                       b_pvc_cotula_var / a_pvc_cotula_var],
                     positions=[0, 1],
                     widths=box_width,
                     boxprops={'linewidth': box_lw, 'color': 'black'},
                     medianprops={'marker': box_marker, 'markersize': 7, 'color': 'black'},
                     whiskerprops={'linewidth': box_whisker_lw, 'color': 'black'},
                     showcaps=True,
                     showfliers=box_showfliers,
                     whis=0)
var_comp2_ax.set_yscale('log')
var_comp2_ax.set_yticks([1e-2, 1e-1])
var_comp2_ax.set_ylim([1e-3, 1e-1])
var_comp2_ax.set_xticks([])
var_comp2_ax.tick_params(labelsize=15)
var_comp2_ax.spines['top'].set_visible(False)
var_comp2_ax.spines['right'].set_visible(False)
var_comp2_ax.set_title(bold_text('PVC'), fontsize=15, pad=35)
var_comp2_ax.set_xticks([0, 1])
var_comp2_ax.tick_params(labelsize=15)
var_comp2_ax.set_xticklabels(bold_text(['CoTu', 'CoTuLa']), fontsize=15, ha='left', rotation=-25)


mplego.labels.add_significance_label(var_comp_ax, bounds=[0.01, 1], which='top', label='***')
mplego.labels.add_significance_label(var_comp2_ax, bounds=[0.01, 1], which='top', label='***')

# Add subplot labels
mplego.labels.apply_subplot_labels(
    axes=[pvc_tuning_plot, ecog_tuning_plot],
    y=1.2,
    labels=['a', 'd'],
    bold=subplot_label_bold,
    size=subplot_label_size)
mplego.labels.apply_subplot_labels(
    axes=[pvc_scatter, pvc_coupling, ecog_scatter, ecog_coupling],
    labels=['b', 'c', 'e', 'f'],
    bold=subplot_label_bold,
    size=subplot_label_size)
mplego.labels.apply_subplot_labels(
    axes=[var_frac_ax],
    x=-0.03,
    y=1.3,
    labels=['g'],
    bold=subplot_label_bold,
    size=subplot_label_size)
mplego.labels.apply_subplot_labels(
    axes=[var_comp_ax, var_comp2_ax],
    x=-0.63,
    y=1.30,
    labels=['h', 'i'],
    bold=subplot_label_bold,
    size=subplot_label_size)

# Save figure
plt.savefig('figure4.pdf', bbox_inches='tight')

In [ ]:
wilcoxon(b_ecog_cotu_var / a_ecog_cotu_var, b_ecog_cotula_var / a_ecog_cotula_var)

In [ ]:
wilcoxon(b_pvc_cotu_var / a_pvc_cotu_var, b_pvc_cotula_var / a_pvc_cotula_var)